In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Define the Trend Following strategy
def trend_follow_strategy(data):
    df = pd.DataFrame(index=data.index)
    df['price'] = data['Close']
    
    df['Returns'] = df['price'].pct_change()

    # Set threshold for positive returns (adjust as needed)
    threshold = 0.01  # Example threshold of 1%

    # Generate signals
    df['Signal_Trend'] = 0  # 0: No signal, 1: Buy, -1: Sell
    df.loc[df['Returns'] > threshold, 'Signal_Trend'] = 1  # Buy signal
    df.loc[df['Returns'] < -threshold, 'Signal_Trend'] = -1 # Sell Signal

    # Calculate the strategy returns
    df['strategy_returns_trend'] = df['Signal_Trend'].shift(1) * df['Returns']
    return df

# Define the MACD 50/200 strategy
def sma_50_200_strategy(data):
    df = pd.DataFrame(index=data.index)
    df['price'] = data['Close']
    
    # Calculate 50-day and 200-day simple moving averages
    df['SMA50'] = df['price'].rolling(window=50).mean()
    df['SMA200'] = df['price'].rolling(window=200).mean()

    # Generate signals
    df['Signal_MACD'] = 0  # 0: No signal, 1: Buy, -1: Sell
    df.loc[df['SMA50'] > df['SMA200'], 'Signal_MACD'] = 1  # Buy signal
    df.loc[df['SMA50'] < df['SMA200'], 'Signal_MACD'] = -1  # Sell signal
    
    # Calculate the daily returns
    df['Returns'] = df['price'].pct_change()
    
    # Calculate the strategy returns
    df['strategy_returns_macd'] = df['Signal_MACD'].shift(1) * df['Returns']
    
    return df

# Define the Bollinger Bands strategy
def bollinger_bands_strategy(data, window=20, num_std=2):
    signals = pd.DataFrame(index=data.index)
    signals['price'] = data['Close']
    
    # Calculate the rolling mean and standard deviation
    signals['rolling_mean'] = data['Close'].rolling(window=window).mean()
    signals['upper_band'] = signals['rolling_mean'] + (num_std * data['Close'].rolling(window=window).std())
    signals['lower_band'] = signals['rolling_mean'] - (num_std * data['Close'].rolling(window=window).std())
    
    # Create buy and sell signals
    signals['signal'] = 0  # 0: Hold, 1: Buy, -1: Sell
    
    # Buy signal: Price crosses below lower Bollinger Band
    signals.loc[signals['price'] < signals['lower_band'], 'signal'] = 1
    
    # Sell signal: Price crosses above upper Bollinger Band
    signals.loc[signals['price'] > signals['upper_band'], 'signal'] = -1
    
    # Calculate the daily returns
    signals['daily_returns'] = data['Close'].pct_change()
    
    # Calculate the strategy returns
    signals['strategy_returns_bollinger'] = signals['signal'].shift(1) * signals['daily_returns']
    
    return signals

# Fetch historical price data from Yahoo Finance API
def fetch_historical_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    return data


In [6]:
# Define the stock symbol, start date, and end date
symbol = 'AAPL'  # Replace with your desired stock symbol
start_date = '2022-01-01'
end_date = '2024-01-01'

# Fetch historical data
data = fetch_historical_data(symbol, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [7]:
# Apply the three strategies
signals_trend = trend_follow_strategy(data)
signals_macd = sma_50_200_strategy(data)
signals_bollinger = bollinger_bands_strategy(data)

# Combine the signals
combined_signals = pd.concat([signals_trend, signals_macd, signals_bollinger], axis=1)

# Calculate combined strategy returns
combined_signals['combined_returns'] = (
    signals_trend['strategy_returns_trend']
    + signals_macd['strategy_returns_macd']
    + signals_bollinger['strategy_returns_bollinger']
)

# Calculate cumulative combined strategy returns
cumulative_combined_strategy_returns = (1 + combined_signals['combined_returns']).cumprod()

# Print performance metrics for combined strategies
total_trades_combined = combined_signals['combined_returns'].count()
winning_trades_combined = combined_signals[combined_signals['combined_returns'] > 0]['combined_returns'].count()
losing_trades_combined = combined_signals[combined_signals['combined_returns'] < 0]['combined_returns'].count()
win_rate_combined = (winning_trades_combined / total_trades_combined) * 100
average_daily_return_combined = combined_signals['combined_returns'].mean()
cumulative_return_combined = cumulative_combined_strategy_returns[-1] - 1

print("======= Combined Strategy Performance =======")
print(f"Total Trades: {total_trades_combined}")
print(f"Winning Trades: {winning_trades_combined}")
print(f"Losing Trades: {losing_trades_combined}")
print(f"Win Rate: {win_rate_combined:.2f}%")
print(f"Average Daily Return: {average_daily_return_combined:.4f}")
print(f"Cumulative Return Combined: {cumulative_return_combined:.4f}")
print("============================================")

======= Combined Strategy Performance =======
Total Trades: 500
Winning Trades: 188
Losing Trades: 163
Win Rate: 37.60%
Average Daily Return: 0.0005
Cumulative Return Combined: 0.1265
